In [ ]:
#!pip install ibmos2spark

# Working with IBM COS in PySpark Notebook

### setup

In [ ]:
# spark
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DoubleType, FloatType, LongType,StructType,StructField
from pyspark.sql import SparkSession,SQLContext, Window, DataFrame
from pyspark.rdd import RDD
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark import StorageLevel

TOTAL_SPARK_CPU_CORES = 4
SQL_SHUFFLE_PARTITIONS_SIZE=1*TOTAL_SPARK_CPU_CORES

# create spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
spark

In [ ]:
import ibmos2spark

myconfig={
    'ACCESS_KEY_ID':'enter access id here',
    'SECRET_ACCESS_KEY':'enter secret here',
    'COS_ENDPOINT':'https://s3.us-east.cloud-object-storage.appdomain.cloud',
}

# connect to COS
credentials = {
  'endpoint': myconfig['COS_ENDPOINT'],  
  'access_key': myconfig['ACCESS_KEY_ID'],
  'secret_key': myconfig['SECRET_ACCESS_KEY']
}
configuration_name = 'mycos_config'  #you can give any string you like

spark_cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name=configuration_name)  

### Read csv file

In [ ]:
# read file to dataframe
test_filename="country-codes_csv.csv"
test_bucket="my-codeset"    
read_sdf = spark.read.option("header","true").csv(spark_cos.url(test_filename,test_bucket))
print("records: ",read_sdf.count())
read_sdf.limit(10).show(10)

### Write file to Parquet format

In [ ]:
# read file to dataframe
save_filename="country-codes_csv.parquest"
test_bucket="my-codeset"    
read_sdf.write.parquet(spark_cos.url(save_filename, test_bucket), mode='overwrite')
print("done")

### Retrive saved parquet file

In [ ]:
# read file to dataframe
read_sdf2 = spark.read.parquet(spark_cos.url(save_filename,test_bucket))
print("records: ",read_sdf2.count())
read_sdf2.limit(10).show(10)